# Imports

In [86]:
import pandas as pd
import numpy as np
import random
import operator

Read the CSV file

In [87]:
# Replace file name/path if needed
data = pd.read_csv('q6_example.csv')

# Extract the voter preferences into a matrix
preferences_matrix = data.iloc[:, 1:].values

# Extract the number of voters for each set of preferences
num_voters = data['Voters'].values

# show the preferences 
print(preferences_matrix)

# show the number of voters
print(num_voters)

[['b' 'a' 'c' 'd' 'e' 'f' 'g']
 ['a' 'c' 'd' 'b' 'e' 'f' 'g']
 ['a' 'd' 'b' 'c' 'e' 'f' 'g']
 ['c' 'a' 'd' 'b' 'e' 'f' 'g']
 ['d' 'c' 'a' 'b' 'e' 'f' 'g']]
[ 6 12 13 10  8]


In [88]:
# sum the number of total votes
total_voters = np.sum(num_voters)
print (f"total number of voters: {total_voters}")

total number of voters: 49


In [89]:
# get the number of candidates
total_candidites = len(preferences_matrix[0])
print (f"total number of voters: {total_candidites}")

total number of voters: 7


In [90]:
# get list of condidates, orderd alphapatically! 
candidates_list = set(preferences_matrix[0])
candidates_list

{'a', 'b', 'c', 'd', 'e', 'f', 'g'}

In [91]:
def SimpleMajorityRulefor2(preferences_matrix, num_voters, candidate_a, candidate_b):
    
    '''
    In this Simple Majority Rule, 
        1) we don't take order of preferences of other candidates 
        2) we don't check absolute majority (> 50% of the votes)
        3) in case of the ties, we define what is the rule to handle it (in our case, candidate A will be winner)
     
    Params:
         a) preferences_matrix: <numpy.ndarray> 
                 Description:    a matrix that defines the preferences for example:
                                 [['a' 'b' 'c' 'd']
                                 ['a' 'c' 'b' 'd']
                                 ['d' 'b' 'a' 'c']
                                 ['d' 'b' 'c' 'a']
                                 ['c' 'b' 'a' 'd']
                                 ['d' 'c' 'b' 'a']], where rows represent ordered voting preference
                             
        b) num_voters: <numpy.ndarray>
                Description:    represent a vector for number of voters for each preference, for example:
                                [5 4 2 6 8 2], where each item represnt the number of voters in 
                                each preference, in our example it means 5 votes for preference ['a' 'b' 'c' 'd']
                                
        c) candidate_a: <string> 
                Description:    the first candidate to check his votes, for example 'a', 'b', ...etc.
                
        d) candidate_b: <string> 
                Description:    the second candidate to check his votes, for example 'a', 'b', ...etc.
        
    '''
    # confirm the candidates are in the list
    if candidate_a not in set(preferences_matrix[0]) or candidate_b not in set(preferences_matrix[0]):
        return "Candidates are not on the voting matrix, no winner", -1, -1 
    
    # initialize the counters 
    a_count, b_count = 0, 0
    
    # idx for the voters counter 
    idx = 0 
    
    # loop over the preference matrix
    for preference in preferences_matrix:
        
        # Find the indices of the two candidates in the preferences matrix
        a_idx = np.where(preference == candidate_a)[0][0]
        b_idx = np.where(preference == candidate_b)[0][0]

        # check if a_idx is higher b_idx then, b is prefered over a
        if a_idx > b_idx:
            
            # increase the counter for b
            b_count += num_voters[idx]
        
        # here we have b_idx > a_idx, then a is prefered over b
        else:
            a_count += num_voters[idx]

        # increment the idx for the voters in general to keep track of voters
        idx += 1

    # based on the definition, if we have tie, then A is return as winner        
    if a_count >= b_count:
        return candidate_a, candidate_b, a_count, b_count
    elif b_count > a_count:
        return candidate_b, candidate_a, b_count, a_count

In [92]:
# Candidates to compare
candidate_a = 'a'
candidate_b = 'b'

# Calculate the winner between 'a' and 'b'
simple_majority_winner, simple_majority_loser, simple_majority_winner_count, simple_majority_loser_count = SimpleMajorityRulefor2(preferences_matrix, 
                                                                                                                                    num_voters, 
                                                                                                                                    candidate_a, 
                                                                                                                                    candidate_b)

# print(f"The winner according to Simple Majority Rule between\n\tcandidate_a: {candidate_a}," +
#       f" with {candidate_a_count} votes,\n\tcandidate_b: {candidate_b}, with {candidate_b_count}"+
#       f" votes,\033[1m\n\tWinner: {simple_majority_winner} \033[0m")

print(f"The winner according to Simple Majority Rule between\n\tcandidate_1: {simple_majority_winner}," +
      f" with {simple_majority_winner_count} votes,\n\tcandidate_2: {simple_majority_loser}, with {simple_majority_loser_count}"+
      f" votes,\033[1m\n\tWinner: {simple_majority_winner} \033[0m")

The winner according to Simple Majority Rule between
	candidate_1: a, with 43 votes,
	candidate_2: b, with 6 votes,
	Winner: a 


In [93]:
def Plurality(preferences_matrix, num_voters):
    '''
    In this function we will implement plurality voting rules
        1) in this rule, we will take care of the first preference only 
        2) it has only one round
        3) we implemented random tie breaking rule in case of ties results
        
    Params: 
         a) preferences_matrix: <numpy.ndarray> 
                 Description:    a matrix that defines the preferences for example:
                                 [['a' 'b' 'c' 'd']
                                 ['a' 'c' 'b' 'd']
                                 ['d' 'b' 'a' 'c']
                                 ['d' 'b' 'c' 'a']
                                 ['c' 'b' 'a' 'd']
                                 ['d' 'c' 'b' 'a']]   , rows represents ordered voting preference
                             
        b) num_voters: <numpy.ndarray>
                Description:    represent a vector for number of voters for each preference, for example:
                                [5 4 2 6 8 2], where each item represnt the number of voters in 
                                each preference, in our example it means 5 votes for preference ['a' 'b' 'c' 'd']
                                
    '''
    # extract the first column in the preferences matrix (first preference)
    first_vote_list = [preference[0] for preference in preferences_matrix]
    
    # initialize a hash table with structure candidate:0 to be used later as counter
    candidates_votes = {item: 0 for item in first_vote_list}
    
    # idx for the number of voters counter
    idx = 0 
    
    # loop over the votes, and count the votes per candidate
    for vote in first_vote_list:
        # update the number of voters for each candidate from the hash table
        candidates_votes[vote] = candidates_votes.get(vote) + num_voters[idx]
        
        # increase the voters index number
        idx += 1
        
    # extract highest number of votes in the hash table   
    max_value = max(candidates_votes.values())
    
    # group all candidate who has number of votes == max number (to handle ties)
    candidates_with_max_value = [candidate for candidate, value in candidates_votes.items() if value == max_value]
    
    # return the candidate with highest vote (randomized if ties)
    return random.choice(candidates_with_max_value), max_value

In [94]:
plurality_winner, plurality_winner_votes_number = Plurality(preferences_matrix, num_voters)
print(f"The winner according to Plurality Rule \033[1m\n\tWinner: {plurality_winner} \033[0m with {plurality_winner_votes_number} votes")

The winner according to Plurality Rule 
	Winner: a  with 25 votes


In [95]:
def PluralityRunOff(preferences_matrix, num_voters):
    '''
    In this function we will implement plurality run off voting rules
        1) in this rule, we will take care of the first preference only 
        2) it has two rounds
        3) we implemented random tie breaking rule in case of ties results
        
    Params: 
         a) preferences_matrix: <numpy.ndarray> 
                 Description:    a matrix that defines the preferences for example:
                                 [['a' 'b' 'c' 'd']
                                 ['a' 'c' 'b' 'd']
                                 ['d' 'b' 'a' 'c']
                                 ['d' 'b' 'c' 'a']
                                 ['c' 'b' 'a' 'd']
                                 ['d' 'c' 'b' 'a']]   , rows represents ordered voting preference
                             
        b) num_voters: <numpy.ndarray>
                Description:    represent a vector for number of voters for each preference, for example:
                                [5 4 2 6 8 2], where each item represnt the number of voters in 
                                each preference, in our example it means 5 votes for preference ['a' 'b' 'c' 'd']
                                
    '''
    # extract the first column in the preferences matrix (first preference)
    first_vote_list = [preference[0] for preference in preferences_matrix]
    
    # initialize a hash table with structure candidate:0 to be used later as counter
    candidates_votes = {item: 0 for item in first_vote_list}
    
    # idx for the number of voters counter
    idx = 0 
    
    # loop over the votes, and count the votes per candidate
    for vote in first_vote_list:

        # update the number of voters for each candidate from the hash table
        candidates_votes[vote] = candidates_votes.get(vote) + num_voters[idx]
        
        # increase the voters index number
        idx += 1
        
    # extract highest number of votes in the hash table   
    max_value = max(candidates_votes.values())

    # ordering the candidate votes in descending order by the number of votes
    candidates_votes = dict(sorted(candidates_votes.items(), key=operator.itemgetter(1), reverse = True))
    
    # get total number of voters:
    total_voters = np.sum(num_voters)
    
    # group all candidate who has number of votes >= 0.5*max number to proceed to round two
    candidates_winning_first_round = [candidate for candidate, value in candidates_votes.items() if value >= 0.5*total_voters]
    
    candidates_with_max_value = []
    # check if the winner has more than 50%, then he wins, in case of ties, random selection 
    if candidates_winning_first_round:
        # group all candidate who has number of votes == max number (to handle ties)
        candidates_with_max_value = [candidate for candidate, value in candidates_votes.items() if value == max_value]
    
        # return the candidate with highest vote (randomized if ties)
        return random.choice(candidates_with_max_value), max_value
    
    candidates_for_second_round = list(dict(sorted(candidates_votes.items(), key=operator.itemgetter(1), reverse = True)).keys())
    print(f"No one has votes >50%, going with the second round with candidates_a: {candidates_for_second_round[0]}, and candidate_b: {candidates_for_second_round[1]}")
    
    # if no one has more than 50%, then do second round        
    candidate_winning_round_two, candidate_losing_round_two, winner_count, loser_count = SimpleMajorityRulefor2(preferences_matrix, num_voters, candidates_for_second_round[0], candidates_for_second_round[1])
    
    return candidate_winning_round_two, winner_count

In [96]:
plurality_runoff_winner, plurality_runoff_winner_votes_number = PluralityRunOff(preferences_matrix, num_voters)
print(f"The winner according to Plurality RunOff Rule \033[1m\n\tWinner: {plurality_runoff_winner} \033[0m with {plurality_runoff_winner_votes_number} votes")

The winner according to Plurality RunOff Rule 
	Winner: a  with 25 votes


In [97]:
def CordocetVoting(preferences_matrix, num_voters):
    '''
    In this function we will implement Condorcet voting rule
        1) in this rule, we will take care of the first preference only 
        2) it has one rounds
        3) we implemented random tie breaking rule in case of ties results
        
    Params: 
         a) preferences_matrix: <numpy.ndarray> 
                 Description:    a matrix that defines the preferences for example:
                                 [['a' 'b' 'c' 'd']
                                 ['a' 'c' 'b' 'd']
                                 ['d' 'b' 'a' 'c']
                                 ['d' 'b' 'c' 'a']
                                 ['c' 'b' 'a' 'd']
                                 ['d' 'c' 'b' 'a']]   , rows represents ordered voting preference
                             
        b) num_voters: <numpy.ndarray>
                Description:    represent a vector for number of voters for each preference, for example:
                                [5 4 2 6 8 2], where each item represnt the number of voters in 
                                each preference, in our example it means 5 votes for preference ['a' 'b' 'c' 'd']
                                
    '''
    # extract the first column in the preferences matrix (first preference)
    candidates = sorted(preferences_matrix[0])

    for candidate1 in candidates:
        
        condorcet = True

        for candidate2 in candidates:
            if candidate1 == candidate2:
                continue
            
            # initialize the counters 
            candidate1_count, candidate2_count = 0, 0
            
            # idx for the voters counter 
            idx = 0 
            
            # loop over the preference matrix
            for preference in preferences_matrix:
                
                # Find the indices of the two candidates in the preferences matrix
                a_idx = np.where(preference == candidate1)[0][0]
                b_idx = np.where(preference == candidate2)[0][0]

                # check if a_idx is higher b_idx then, b is prefered over a
                if a_idx > b_idx:
                    
                    # increase the counter for b
                    candidate2_count += num_voters[idx]
                
                # here we have b_idx > a_idx, then a is prefered over b
                else:
                    candidate1_count += num_voters[idx]

                # increment the idx for the voters in general to keep track of voters
                idx += 1

            if candidate1_count < candidate2_count:
                condorcet = False
                break

        if condorcet:
            return candidate1, candidate1_count
        
    return None, 0

In [98]:
cordocet_winner, cordocet_winner_votes = CordocetVoting(preferences_matrix, num_voters)
if cordocet_winner is not None:
    print(f"The winner according to Cordocet Voting Rule \033[1m\n\tWinner: {cordocet_winner} \033[0m with {cordocet_winner_votes} votes")
else:
    print("There is no Cordocet winner for this list of preferences.")

The winner according to Cordocet Voting Rule 
	Winner: a  with 49 votes


In [105]:
def BordaVoting(preferences_matrix, num_voters):
    '''
    In this function we will implement Borda Voting rule
        1) in this rule, the preferences of voters are indexed in ascending order: [preference1 (index1) > preference2 (index2), etc.]
        2) candidate with minimum votes is returned at the end
        3) it has only one round
        4) in case of ties, random selection 
        
    Params: 
         a) preferences_matrix: <numpy.ndarray> 
                 Description:    a matrix that defines the preferences for example:
                                 [['a' 'b' 'c' 'd']
                                 ['a' 'c' 'b' 'd']
                                 ['d' 'b' 'a' 'c']
                                 ['d' 'b' 'c' 'a']
                                 ['c' 'b' 'a' 'd']
                                 ['d' 'c' 'b' 'a']]   , rows represents ordered voting preference
                             
        b) num_voters: <numpy.ndarray>
                Description:    represent a vector for number of voters for each preference, for example:
                                [5 4 2 6 8 2], where each item represnt the number of voters in 
                                each preference, in our example it means 5 votes for preference ['a' 'b' 'c' 'd']
                                
    '''   

    # initialize a hash table with structure candidate:0 to be used later as counter
    borda_score = {item: 0 for item in preferences_matrix[0]}

    # initialize a counter 
    idx = 0

    # evaluate borda scores for each candidate
    for preference in preferences_matrix:
        for index, candidate in enumerate(preference):
            borda_score[candidate] += (index + 1) * num_voters[idx]
        idx += 1

    # handle ties in case it exists 
    min_value = min(borda_score.items(), key=operator.itemgetter(1))[1]
    # group all candidate who has number of votes == max number (to handle ties)
    candidates_with_min_value = [candidate for candidate, value in borda_score.items() if value == min_value]
    
    # return the candidate with highest vote (randomized if ties)
    return random.choice(candidates_with_min_value), min_value

In [106]:
borda_winner, borda_winner_votes_number = BordaVoting(preferences_matrix, num_voters)
print(f"The winner according to Borda Voting Rule \033[1m\n\tWinner: {borda_winner} \033[0m with {borda_winner_votes_number} votes")

{'b': 165, 'a': 81, 'c': 120, 'd': 124, 'e': 245, 'f': 294, 'g': 343}
The winner according to Borda Voting Rule 
	Winner: a  with 81 votes


In [101]:
def check_percentage_best_candidate(preferences_matrix, num_voters):
    # Initialize a dictionary to store the number of voters for each candidate
    candidate_votes = {}

    # Loop through the voters and preferences to count the number of voters for each candidate
    for i in range(len(num_voters)):
        best_candidate = preferences_matrix[i][0]
        if best_candidate not in candidate_votes:
            candidate_votes[best_candidate] = 0
        candidate_votes[best_candidate] += num_voters[i]

    # Find the maximum number of votes
    max_num_voters = max(candidate_votes.values())

    # Calculate the percentage of voters with the best candidate(s)
    percentage_same_best = (max_num_voters / np.sum(num_voters)) * 100

    return percentage_same_best

In [102]:
def check_percentage_different_preferences(preferences_matrix, num_voters):
    # Create a set to store unique preference combinations
    unique_preferences = set()

    num_preferences = preferences_matrix.shape[1]  # Get the number of preferences

    for i in range(len(num_voters)):
        # Convert the voter's preferences to a tuple to make it hashable
        preference_tuple = tuple(preferences_matrix[i])

        # Add the preference combination to the set
        unique_preferences.add(preference_tuple)
    # Calculate the percentage of different preferences
    percentage_different_preferences = (len(unique_preferences) / np.sum(num_voters)) * 100

    return percentage_different_preferences

In [103]:
percentage_same_best = check_percentage_best_candidate(preferences_matrix, num_voters)
print(f"Percentage of voters with the same 'best candidate': {percentage_same_best}%")

Percentage of voters with the same 'best candidate': 51.02040816326531%


In [104]:
percentage_different = check_percentage_different_preferences(preferences_matrix, num_voters)
print(f"Percentage of voters with different preferances: {percentage_different}%")

Percentage of voters with different preferances: 10.204081632653061%
